In [1]:
import matplotlib.colors as mcolors
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer

from interpreto.attributions.methods import IntegratedGradients, OcclusionExplainer, Saliency, SmoothGrad
from interpreto.commons.granularity import GranularityLevel
from interpreto.visualizations.attributions.classification_highlight import (
    GenerationAttributionVisualization,
    MultiClassAttributionVisualization,
    SingleClassAttributionVisualization,
)

/home/fanny.jourdan/dev/interpreto/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/fanny.jourdan/dev/interpreto/interpreto/attributions/aggregations/linear_regression_aggregation.py:130: UserWarning: As of jaxtyping version 0.2.24, jaxtyping now prefers the syntax
```
from jaxtyping import jaxtyped
# Use your favourite typechecker: usually one of the two lines below.
from typeguard import typechecked as typechecker
from beartype import beartype as typechecker

@jaxtyped(typechecker=typechecker)
def foo(...):
```
and the old double-decorator syntax
```
@jaxtyped
@typechecker
def foo(...):
```
should no longer be used. (It will continue to work as it did before, but the new approach will produce more readable error messages.)
In particular note that `typechecker` must be passed via keyword arg

Currents:
- Occlusion
- Saliency
- Integrated Gradients
- SmoothGrad

Soon:
- LIME
- KernelSHAP
- Sobol



# Classification task

In [2]:
model_name = "textattack/bert-base-uncased-imdb"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Occlusion 

In [ ]:
explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

attribution_outputs = explainer.explain(
    model_inputs=[
        "This is the best movie I have ever seen. The cinematography was uncharacteristically breathtaking."
    ],
    mode="logits",
)

viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

# mode in {"logits", "softmax", "log_softmax"}

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [4]:
explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

list_attribution_outputs = explainer.explain(
    model_inputs=[
        "This is the best movie I have ever seen.",
        "I hate this movie.",
        "This movie is super good. I love it.",
    ],
    mode="logits",
)

for attribution_outputs in list_attribution_outputs:
    viz = SingleClassAttributionVisualization(
        attribution_output=attribution_outputs,
        color=mcolors.to_rgb("red"),
        css=".common-word-style { margin-right: 0.3em }",
    )
    viz.display()

In [5]:
explainer = OcclusionExplainer(
    model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.TOKEN
)

attribution_outputs = explainer.explain(
    model_inputs="This is the best movie I have ever seen.",
    targets=[0, 1],
    mode="logits",
)

viz = MultiClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    class_colors=[mcolors.to_rgb("green"), mcolors.to_rgb("blue")],
    class_names=["negative review", "positive review"],
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

## Saliency

In [37]:
explainer = Saliency(model=model, batch_size=4, tokenizer=tokenizer)

attribution_outputs = explainer.explain(
    model_inputs=["This is the best movie I have ever seen."],
)

viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

## Integrated Gradient

In [25]:
explainer = IntegratedGradients(model=model, tokenizer=tokenizer, batch_size=4, n_interpolations=10)

attribution_outputs = explainer.explain(model_inputs="This is the best movie I have ever seen.")

viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

## SmoothGrad

In [26]:
explainer = SmoothGrad(model=model, batch_size=4, tokenizer=tokenizer, n_interpolations=50, noise_level=0.01)

attribution_outputs = explainer.explain(
    model_inputs=["I love this movie"],
)

viz = SingleClassAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("red"),
    css=".common-word-style { margin-right: 0.3em }",
)
viz.display()

# Generation task

In [38]:
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

## Occlusion

In [39]:
explainer = OcclusionExplainer(model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.WORD)

attribution_outputs = explainer.explain(
    model_inputs="Hi there, how are you?", mode="softmax", generation_kwargs={"max_length": 10}
)

viz = GenerationAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("orange"),
    highlight_border=False,
    normalize=True,
)
viz.display()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/fanny.jourdan/dev/interpreto/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [29]:
explainer = OcclusionExplainer(
    model=model, batch_size=4, tokenizer=tokenizer, granularity_level=GranularityLevel.ALL_TOKENS
)

attribution_outputs = explainer.explain(
    model_inputs="Hi there, how are you?",
    targets="I am fine, thank you",
    mode="softmax",
    generation_kwargs={"max_length": 10},
)

viz = GenerationAttributionVisualization(
    attribution_output=attribution_outputs[0],
    color=mcolors.to_rgb("orange"),
    highlight_border=False,
    normalize=True,
)
viz.display()

## Integrated Gradients

In [40]:
explainer = IntegratedGradients(model=model, tokenizer=tokenizer, batch_size=4, n_interpolations=10)

list_attribution_outputs = explainer.explain(
    model_inputs=["Hi there, how are you?", "What time is it?"], generation_kwargs={"max_length": 10}
)

for attribution_outputs in list_attribution_outputs:
    viz = GenerationAttributionVisualization(
        attribution_output=attribution_outputs,
        color=mcolors.to_rgb("orange"),
        highlight_border=False,
        normalize=True,
    )
    viz.display()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
